In [ ]:
###import packages
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import (f_classif,SelectKBest)
import scipy.stats as stats
import warnings
import sys
sys.path.insert(0, '/libraries')
from libraries import Imputation_library
import os
os.environ["PYTHONHASHSEED"] = "0"

In [ ]:
###user-defined parameters
is_balanced = 0 #do we want to use weights during classification?
is_median = 1 #do we want to use median or mean when select optimal n (number of features)
Dataset_N = 1 #number of the dataset

In [ ]:
###standard parameters
level = 0.5 #auc when guess is random
test = 0.2 #fraction for testing
val = 0.2 #fraction for validation (from the rest part)
ITER = 5 #number of iteration for random forest
Imputation_sets = 5 #number of imputation sets
n_features = 20 #the maximum number of features we consider

In [ ]:
test_sets = int(1/test) #number of testing sets (5)
val_sets = int(1/val) #number of validation sets (5)
if Dataset_N==1:
    binary = 11 #threshold for depression
if Dataset_N==2:
    is_sparse = 1 #do we want to reduce sparsity?
    sparsity_level = 0.9 #sparsity threshold to merge bacteria together
if is_balanced == 1:
    class_weight = "balanced"
else:
    class_weight = None

In [ ]:
###metadata
if Dataset_N==1:
    Metadata=pd.read_csv("BASIC_metadata_full.csv",sep=',',low_memory=False) #read file with depression levels and ids of participants
    Metadata.loc[Metadata.TimePoint=="Trimester2","TimePoint"] = 0 #timepoits are 0,1,2
    Metadata.loc[Metadata.TimePoint=="Trimester3","TimePoint"] = 1
    Metadata.loc[Metadata.TimePoint=="PostpartumWeek6","TimePoint"] = 2
    i = Metadata[Metadata.ReadsNumber<500000].index #remove insufficient reads
    #Metadata.loc[i, 'ReadsNumber'] = np.nan
    EPDS = 2*np.ones_like(Metadata.EPDS) #2 is for missingness
    EPDS[Metadata.EPDS>binary] = 1 #binary labels for depression
    EPDS[Metadata.EPDS<=binary] = 0
if Dataset_N==2:
    Metadata=pd.read_csv("GMAP_metadata_public.csv",sep=",",low_memory=False)
    Metadata = Metadata.rename(columns={"sample": "tp"})
    Metadata.loc[Metadata.tp=="3_twomonth","tp"] = 0
    Metadata.loc[Metadata.tp=="4_fourmonth","tp"] = 1
    Metadata.loc[Metadata.tp=="5_sixmonth","tp"] = 2
    time = [".3.twomonth", ".4.fourmonth", ".5.sixmonth"]
    i = Metadata[pd.Series.isna(Metadata.case_id)].index #remove NaN values
    Metadata = Metadata.drop(i)
    AP = np.zeros_like(Metadata.case_id)
    AP[Metadata.case_id.to_numpy()=="AP Case"] = 1

In [ ]:
###data
profile =pd.read_csv("Species_Profile_full.csv",sep=',',low_memory=False) #read file with compositional data
species=profile.to_numpy()[:,1:]/100 #normilized to 1
full_list_bacteria = list(profile.columns)[1:]
Where = lambda x: np.where(np.array(full_list_bacteria) == x)[0][0]
arg_bacteria = [Where("Veillonella_parvula"), Where("Haemophilus_parainfluenzae")]

In [ ]:


d_full = np.shape(species)[1]
X_full, X_0, X_1, labels, labels_0, labels_1 = [[] for _ in range(6)] #X is data in 2 rows, labels are EPDS at tp 2
ID = profile.Sample_id.to_numpy() #id of a sample in profile
Individual_ID = Metadata.Individual_ID #id of a person
for i in np.unique(Individual_ID):
    TP = Metadata.TimePoint[Individual_ID == i].to_numpy()
    if sum((TP - np.array([0,1,2]))**2)!=0:
        warnings.warn("TP are missing or in incorrect order") #does not happen in the dataset we have
    Outcomes = EPDS[Individual_ID == i]
    if Outcomes[2]!=2:
        Reads = Metadata.ReadsNumber[Individual_ID == i].to_numpy()
        Sample_ID = Metadata.Sample_ID[Individual_ID == i] #id of a sample in metadata
        if 1-np.isnan(Reads[0]) and 1-np.isnan(Reads[1]): #complete data
            labels = np.append(labels, Outcomes[2])
            Dataset_line = np.zeros((1,2,d_full))
            for j in range(2):
                tp_id = Sample_ID[TP == j].to_numpy()
                Dataset_line[:, j, :] = species[np.where(ID == tp_id)[0], :]
            X_full = Dataset_line if np.size(X_full) == 0 else np.concatenate([X_full, Dataset_line])
        if 1-np.isnan(Reads[0]) and np.isnan(Reads[1]): #tp 1 is missing
            labels_0 = np.append(labels_0, Outcomes[2])
            Dataset_line = np.zeros((1,2,d_full))
            tp_id = Sample_ID[TP == 0].to_numpy()
            Dataset_line[:, 0, :] = species[np.where(ID == tp_id)[0], :]
            X_0 = Dataset_line if np.size(X_0) == 0 else np.concatenate([X_0, Dataset_line])
        if np.isnan(Reads[0]) and 1-np.isnan(Reads[1]): #tp 0 is missing
            labels_1 = np.append(labels_1, Outcomes[2])
            Dataset_line = np.zeros((1,2,d_full))
            tp_id = Sample_ID[TP == 1].to_numpy()
            Dataset_line[:, 1, :] = species[np.where(ID == tp_id)[0], :]
            X_1 = Dataset_line if np.size(X_1) == 0 else np.concatenate([X_1, Dataset_line])
X_f, X_t = X_full[labels == 0], X_full[labels == 1] #f = healthy, t = depressed
size_f, size_t = np.shape(X_f)[0], np.shape(X_t)[0]
print("sizes of complete f and t sets:",size_f,size_t)
test_f, test_t = int(size_f*test), int(size_t*test) #amounts to put to test set
print("sizes of test sets",test_f, test_t)
AUC_TEST, N_OPT, SENSITIVITY, SPECIFICITY = [[] for _ in range(4)]
for IS in range(Imputation_sets):
    sample_seed = IS + ITER
    aucs_val = np.zeros((val_sets, ITER, n_features))
    for TS in range(test_sets): #test sets
        print("test group", TS)
        test_f_range, test_t_range = range(TS * test_f, (TS + 1) * test_f), range(TS * test_t, (TS + 1) * test_t)
        rest_f_range, rest_t_range = np.setdiff1d(range(size_f), test_f_range), np.setdiff1d(range(size_t), test_t_range)
        X_f_test, X_t_test = X_f[test_f_range], X_t[test_t_range]
        X_test = np.concatenate([X_f_test,X_t_test])
        label_test = np.zeros(test_f + test_t)
        label_test[test_f:] = 1
        rest_f, rest_t = np.size(rest_f_range), np.size(rest_t_range)
        val_f, val_t = int(rest_f * val), int(rest_t * val) #amounts to put to validation set
        print("sizes of val sets",val_f, val_t)
        train_f, train_t = rest_f - val_f, rest_t - val_t #the rest of rest is for training set
        print("sizes of train sets", train_f, train_t)
        for VS in range(val_sets): #val sets
            print("val group", VS)
            val_f_range, val_t_range = rest_f_range[range(VS* val_f, (VS + 1) * val_f)], rest_t_range[range(VS* val_t, (VS + 1) * val_t)]
            train_f_range, train_t_range = np.setdiff1d(rest_f_range, val_f_range), np.setdiff1d(rest_t_range, val_t_range)
            X_f_val, X_t_val, X_f_train, X_t_train = X_f[val_f_range], X_t[val_t_range], X_f[train_f_range], X_t[train_t_range]
            X_val, X_train = np.concatenate([X_f_val, X_t_val]), np.concatenate([X_f_train, X_t_train])
            label_val, label_train = np.zeros(val_f + val_t), np.zeros(train_f + train_t)
            label_val[val_f:], label_train[train_f:] = 1, 1
            X_test_cut, X_0_cut, X_1_cut = X_test, X_0, X_1
            X = [X_train, X_val, X_test_cut, X_0_cut, X_1_cut]
            d = np.shape(X[0])[2]
            delta = np.min(X[0][X[0] > 0])
            print("delta", delta)  # the smallest element
            for xi in range(len(X)):
                X[xi] = (X[xi] + delta) / (1 + delta * d) #remove 0s
            X_train, X_val, X_test_cut, X_0_cut, X_1_cut = X
            ###classes
            classes_train, classes_val, classes_test = np.ones((np.shape(X_train)[0],1)),np.ones((np.shape(X_val)[0],1)),np.ones((np.shape(X_test_cut)[0],1))
            classes_0 = np.zeros((np.shape(X_0_cut)[0], 1))
            #classes_1 = np.zeros((np.shape(X_1_cut)[0], 1))
            ###imputation
            X_0_cut = Imputation_library.impute(X_train, X_val, X_0_cut, 0, 1, sample_seed, arg_bacteria)
            # X_1_cut = Imputation_library.impute(X_train, X_val, X_1_cut, 1, 0, sample_seed, arg_bacteria)
            ###
            X_train, classes_train = np.concatenate([X_train, X_0_cut]), np.concatenate([classes_train, classes_0]) ### concatenate full set with imputed sets
            label_train = np.concatenate([label_train, labels_0])
            ### feature extraction
            stats_train, stats_val = Transformation_library.transform_2(X_train,  arg_bacteria), Transformation_library.transform_2(X_val,  arg_bacteria) ### construct features from 2 bacteria
            stats_train, stats_val  = np.concatenate([stats_train, classes_train] ,axis = -1), np.concatenate([stats_val, classes_val], axis = -1)
            n_features = np.min([n_features, np.shape(stats_train)[1]])
            for j in range(n_features): #choose the best j + 1 features using filter method
                sel = SelectKBest(f_classif, k=j + 1).fit(pd.DataFrame(stats_train), label_train)
                Index = sel.get_support()
                for seed in range(ITER): #classify by random forest with different random seeds
                    rf = RandomForestClassifier(random_state = seed, class_weight=class_weight).fit(stats_train[:,Index], label_train)
                    y_pred = rf.predict(stats_val[:,Index])
                    aucs_val[VS, seed, j] = roc_auc_score(label_val, y_pred)
        if is_median == 1: #optimal number of features by median of mean
            print("medians", np.median(aucs_val, axis=(0, 1))) #average by validation sets and random seeds
            n_opt = np.argmax(np.median(aucs_val, axis=(0, 1))) + 1
        else:
            print("means", np.mean(aucs_val, axis = (0,1)))
            n_opt=np.argmax(np.mean(aucs_val, axis = (0,1))) + 1
        N_OPT =np.append(N_OPT, n_opt)
        stats_rest, label_rest = np.concatenate([stats_train, stats_val]), np.concatenate([label_train,label_val])#concatenate everything except testing set
        stats_test = Transformation_library.transform_2(X_test_cut,  arg_bacteria)
        stats_test = np.concatenate([stats_test, classes_test], axis = -1)
        sel = SelectKBest(f_classif, k=n_opt).fit(pd.DataFrame(stats_rest), label_rest) #select optimal number of features
        Index = sel.get_support()
        print("test features:",sel.get_feature_names_out()) #from features namas we can reconstruct what bacteria we use
        for seed in range(ITER):
            rf = RandomForestClassifier(random_state = seed, class_weight=class_weight).fit(stats_rest[:, Index], label_rest)
            print("importances",rf.feature_importances_) #display importances of features
            y_pred = rf.predict(stats_test[:, Index])
            print("auc", roc_auc_score(label_test, y_pred)) #display balanced accuracy
            AUC_TEST = np.append(AUC_TEST, roc_auc_score(label_test, y_pred)) # balanced accuracy on testing set
            SENSITIVITY = np.append(SENSITIVITY, sum(y_pred[label_test==1]==1)/sum(label_test==1))
            SPECIFICITY = np.append(SPECIFICITY, sum(y_pred[label_test==0]==0)/sum(label_test==0))
print("mean auc",np.mean(AUC_TEST))
result= stats.ttest_1samp(AUC_TEST, level, alternative="greater")
print("p value",result.pvalue)
print("true positive",np.mean(SENSITIVITY))
print("true negative",np.mean(SPECIFICITY))
print("n opt",np.mean(N_OPT))
print(N_OPT)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/envs/imputation/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/imputation/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/envs/imputation/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/imputation/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instan

sizes of complete f and t sets: 190 31
sizes of test sets 38 6
test group 0
sizes of val sets 30 5
sizes of train sets 122 20
val group 0
delta 5e-07
val group 1
delta 5e-07
val group 2
delta 5e-07
val group 3
delta 5e-07
val group 4
delta 5e-07
medians [0.45       0.58333333 0.58333333]
test features: ['x0' 'x1']
importances [0.49928245 0.50071755]
auc 0.4605263157894737
importances [0.49088107 0.50911893]
auc 0.4736842105263158
importances [0.4930051 0.5069949]
auc 0.4473684210526316
importances [0.49791781 0.50208219]
auc 0.4605263157894737
importances [0.49725636 0.50274364]
auc 0.4605263157894737
test group 1
sizes of val sets 30 5
sizes of train sets 122 20
val group 0
delta 4.0000000000000003e-07
val group 1
delta 4.0000000000000003e-07
val group 2
delta 4.0000000000000003e-07
val group 3
delta 4.0000000000000003e-07
val group 4
delta 4.0000000000000003e-07
medians [0.48333333 0.48333333 0.58333333]
test features: ['x0' 'x1' 'x2']
importances [0.49106852 0.4964613  0.01247018]
a

KeyboardInterrupt: 